In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm.auto import tqdm

import captchanet

data_dir = Path('/home/hadim/.data/Neural_Network/captchanet')

dataset_dir = data_dir / 'dataset_v6'
train_data_dir = dataset_dir / 'training'
val_data_dir = dataset_dir / 'validation'

model_dir = data_dir / 'model'
model_name = 'v6'
model_path = model_dir / model_name

tokenizer_path = model_path / "tokenizer.json"

# Get tokenizer
with open(tokenizer_path) as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
  
model = tf.keras.models.load_model(str(model_path / 'model'))

In [3]:
model_image_size = (224, 224)
n  = 8
fnames = [str(p) for p in val_data_dir.glob("*.tfrecord")]
dataset = tf.data.TFRecordDataset(fnames)
dataset = dataset.map(map_func=captchanet.decode_data(tokenizer, image_size=model_image_size, max_len_word=10, input_as_dict=True))
dataset = dataset.shuffle(1024)
dataset = dataset.batch(n)
data = [d for d in dataset.take(1)][0]

images = data['image']
labels = data['label']
words = model(images)

In [41]:
# Define a model that do image preprocessing before inference.
class CaptchaSolverInferenceModel(tf.keras.Model):
  
  def __init__(self, captchanet_model, **kwargs):
    super().__init__(**kwargs)
    self.captchanet_model = captchanet_model

  @tf.function(input_signature=[tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8)])
  def solve(self, images):
    images = tf.cast(images, tf.float32)
    
    # Normalize images
    images = tf.image.per_image_standardization(images)
    #stddev = tf.math.reduce_std(images, axis=0)
    #mean = tf.math.reduce_mean(images, axis=0)
    #images = (images - mean) / stddev
    
    images = tf.image.resize(images, (224, 224))
    
    labels = self.captchanet_model(images)
    #labels = tf.argmax(labels, axis=2)
    # TODO: find a way to integrate the tokenizer.
    
    # Freeze the tokenizer
    #n_letter = max(tokenizer.index_word.keys()) + 1
    #index = [''] * n_letter
    #for k, v in tokenizer.index_word.items():
    #  index[k] = v
    #index = tf.constant(index, dtype=tf.string)
        
    # Decode labels
    #decoder = lambda x: tf.gather_nd(index, tf.expand_dims(x, axis=-1))
    #words = tf.map_fn(decoder, labels, dtype=tf.string)
    #joiner = lambda x: tf.strings.reduce_join(x, axis=0)
    #words = tf.map_fn(joiner, words, dtype=tf.string)

    return images

  
inference_model = CaptchaSolverInferenceModel(model)
inference_model_path = model_path / 'inference_model'
tf.saved_model.save(inference_model, str(inference_model_path))

W0713 19:07:56.051268 140249030874944 saved_model.py:758] Skipping full serialization of Keras model <__main__.CaptchaSolverInferenceModel object at 0x7f8cc811b588>, because its inputs are not defined.
W0713 19:07:56.569405 140249030874944 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.InputLayer object at 0x7f8d58274a20>, because it is not built.
W0713 19:07:56.570856 140249030874944 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.KerasLayer object at 0x7f8d58274b00>, because it is not built.
W0713 19:07:56.572890 140249030874944 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.Dropout object at 0x7f8d58274e10>, because it is not built.
W0713 19:07:56.574497 140249030874944 saved_model.py:765] Skipping full serialization of Keras layer <tensorflow.python.keras.saving.saved_model.Dense object at 0x7f8d5827d0f0>,

In [42]:
import tensorflowjs as tfjs
tfjs.converters.convert_tf_saved_model(str(inference_model_path), '/home/hadim/testmodel')

ValueError: Input 0 of node Func/StatefulPartitionedCall/model/StatefulPartitionedCall/StatefulPartitionedCall/keras_layer/StatefulPartitionedCall/StatefulPartitionedCall_1/input/_1723 was passed float from Func/StatefulPartitionedCall/model/StatefulPartitionedCall/StatefulPartitionedCall/keras_layer/StatefulPartitionedCall/input/_855:0 incompatible with expected resource.